The dataset is [here](https://github.com/google-research/google-research/tree/master/goemotions)

In [ ]:
'''from google.colab import auth

auth.authenticate_user()'''

In [ ]:
'''!curl https://sdk.cloud.google.com | bash'''

In [ ]:
#!gcloud init

In [ ]:
!gsutil cp -r gs://gresearch/goemotions/data/full_dataset/ .

In [ ]:
import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score

Will use goemotion1, goemotion2 to train and goemotion3 to test

In [ ]:
df1 = pd.read_csv("/kaggle/working/full_dataset/goemotions_1.csv")
df2 = pd.read_csv("/kaggle/working/full_dataset/goemotions_2.csv")
df3 = pd.read_csv("/kaggle/working/full_dataset/goemotions_3.csv")

In [ ]:
df1 = df1[df1['example_very_unclear'] == False]
df2 = df2[df2['example_very_unclear'] == False]
df3 = df3[df3['example_very_unclear'] == False]

In [ ]:
df1.shape

In [ ]:
df_train = pd.concat([df1,df2,df3], axis=0)

In [ ]:
df_train.columns

In [ ]:
df_train.info()

In [ ]:
df = df_train.drop(['id','author','subreddit','link_id','parent_id','created_utc','rater_id','example_very_unclear'],axis = 1)

In [ ]:
df.info()

In [ ]:
# Download NLTK resources (run once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.data.path.append("/root/nltk_data")
nltk.download('wordnet')

In [ ]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

In [ ]:
def remove_byte_data(value):
    if isinstance(value, bytes):
        value = value.decode('utf-8', errors='ignore')
        removelist = ""
        value = re.sub('<.*?>', '', value)  # remove HTML tags
        value = re.sub('https://.*', '', value)  # remove URLs
        value = re.sub(r'[^a-zA-Z0-9' + removelist + ']', ' ', result)  # remove non-alphanumeric characters
        value = result.lower()
    else:
        return str(value)
        removelist = ""
        value = re.sub('<.*?>', '', value)  # remove HTML tags
        value = re.sub('https://.*', '', value)  # remove URLs
        value = re.sub(r'[^a-zA-Z0-9' + removelist + ']', ' ', value)  # remove non-alphanumeric characters
        value = result.lower()

df['text'] = df['text'].apply(lambda x: remove_byte_data(x))

In [ ]:
def preprocess_text(text):
    text = str(text)
    # Tokenization
    tokens = word_tokenize(text)

    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]

    # Convert to lowercase
    tokens = [token.lower() for token in tokens]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Example usage
df['text'] = df['text'].apply(lambda x: preprocess_text(x))

In [ ]:
# Create TF-IDF vectorizer
vectorizer = CountVectorizer(max_features = 5500)

# Fit the vectorizer toCountVectorizerpreprocessed text data
X = vectorizer.fit_transform(df['text'])

# Get the feature names (words) as a list
feature_names = vectorizer.get_feature_names_out()

# Print the feature names (optional)
print(feature_names)

# Access the feature vectors
feature_vectors = X.toarray()

# Print the shape of the feature vectors (optional)
print(feature_vectors.shape)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, df[['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']], test_size=0.2, random_state=42)

# Create a Naive Bayes classifier
nb_classifier = MultinomialNB(alpha=1)

#making it 1D
y_train = np.argmax(y_train.values, axis=1)
y_test = np.argmax(y_test.values, axis=1)

# Train the classifier
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
unique_rows= np.unique(y_pred, axis=0)
unique_rows